In [ ]:
'''
Willie Hood & Jamia Mason
Webscraping Script
site: https://www.animefillerlist.com/
'''

#### Create initial Dataframe

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.animefillerlist.com/shows'
url_request = BeautifulSoup(requests.get(url).text)

rough_links = []
rough_names = []
for i in url_request.find_all('a'):
    rough_links.append(i.get('href'))
for i in url_request.find_all('a',href=True):
    rough_names.append(i.string)

clean_links = []
clean_names = []
refrence_url = 'https://www.animefillerlist.com'
for i in range(11,318):
    clean_links.append(refrence_url+rough_links[i])
    clean_names.append(rough_names[i])

import pandas as pd
clean_dict = {'Anime Name': clean_names,
              'Info Links': clean_links}
anime_df = pd.DataFrame(clean_dict)
display(anime_df)

#### Create final dataframe

In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
'''
extend_dataframe: Used to add the correct columns to the original dataframe
data_shaper: used to append the web scraped data to the original dataframe.
show_filter: used to filter through the different attributes on the webpage.
duplicate_removal: used to remove duplicate episodes from the filler and mixed episode counts.
storage_checker: This is used to back the logic in data_shaper function. It ensures no episodes were counted twice in the storages and adds N/A to any storage with no entries.
show_####_filter: used to sort through attributes in the webpages
'''
def extend_dataframe(dataframe):
    new_columns = ['Total Episodes','Manga Canon Eps','Anime Canon Eps',
                   'Filler Eps','Mixed Canon/Filler Eps']
    for i in new_columns:
        dataframe[i] = 'N/A'
    return dataframe

def data_shaper(dataset):
    for show_url in range(len(dataset)):
        manga,anime,mixed,filler = show_filter(BeautifulSoup(requests.get(dataset['Info Links'][show_url]).text))
        dataset['Manga Canon Eps'][show_url] = 'N/A' if manga=='N/A' else len(manga)
        dataset['Anime Canon Eps'][show_url] = 'N/A' if anime=='N/A' else len(anime)
        dataset['Filler Eps'][show_url] = 'N/A' if filler=='N/A' else len(filler)
        dataset['Mixed Canon/Filler Eps'][show_url] = 'N/A' if mixed=='N/A' else len(mixed)

        dataset['Total Episodes'] = dataset['Manga Canon Eps']+dataset['Anime Canon Eps']+dataset['Filler Eps']+dataset['Mixed Canon/Filler Eps']
    return dataset

def show_filter(url_request):
    manga_storage = storage_checker(show_manga_filter(url_request))
    anime_storage = storage_checker(show_anime_filter(url_request))
    mixed_storage = storage_checker(show_mixed_filter(url_request))
    filler_storage = storage_checker(show_filler_filter(url_request))

    if mixed_storage != 'N/A' and filler_storage != 'N/A':
        filler_storage,mixed_storage = duplicate_removal(mixed_storage,filler_storage)

    return manga_storage,anime_storage,mixed_storage,filler_storage

def duplicate_removal(mixed,filler):
    if len(filler) > len(mixed):
        filler = list(set(filler).difference(mixed))
    else:
        mixed = list(set(mixed).difference(filler))
    return filler, mixed

def storage_checker(storage):
    if set(storage) != 0:
        storage = list(set(storage))
    else: storage = 'N/A'
    return storage

def show_manga_filter(url_request):
    manga_storage = []
    for i in url_request.find_all('tr',{'class': re.compile('manga.*')}):
        for ii in i.find_all('td',{'class': re.compile('Number.*')}):
            manga_storage.append(int(ii.next_element))
    return manga_storage

def show_anime_filter(url_request):
    anime_storage = []
    for i in url_request.find_all('tr',{'class': re.compile('anime.*')}):
        for ii in i.find_all('td',{'class': re.compile('Number.*')}):
            anime_storage.append(int(ii.next_element))
    return anime_storage

def show_mixed_filter(url_request):
    mixed_storage = []
    for i in url_request.find_all('tr',{'class': re.compile('mixed.*')}):
        for ii in i.find_all('td',{'class': re.compile('Number.*')}):
            mixed_storage.append(int(ii.next_element))
    return mixed_storage

def show_filler_filter(url_request):
    filler_storage = []
    for i in url_request.find_all('tr',{'class': re.compile('filler.*')}):
        for ii in i.find_all('td',{'class': re.compile('Number.*')}):
            filler_storage.append(int(ii.next_element))
    return filler_storage

In [ ]:
final_anime_df = data_shaper(extend_dataframe(anime_df))
display(final_anime_df)